In [318]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Download the data

In [319]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [320]:
concrete_data.shape

(1030, 9)

In [321]:
#Let's check the dataset for any missing values.
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [322]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

# Split data into predictors and target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [323]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [324]:
#Let's do a quick sanity check of the predictors and the target dataframes.
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [325]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [326]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [327]:
#Let's save the number of predictors to _n_cols_ since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

# Build a Neural Network


In [328]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='Adam', loss='mean_squared_error')
    return model

In [329]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.2, random_state=4)

In [330]:
# build the model
model = regression_model()

# Train and Test the Network


In [331]:
# build the model
model.fit( X_test, y_test,epochs=50, verbose=2)

Epoch 1/50
7/7 - 1s - loss: 2096.2598
Epoch 2/50
7/7 - 0s - loss: 1834.1187
Epoch 3/50
7/7 - 0s - loss: 1670.9686
Epoch 4/50
7/7 - 0s - loss: 1586.2200
Epoch 5/50
7/7 - 0s - loss: 1545.6096
Epoch 6/50
7/7 - 0s - loss: 1522.5714
Epoch 7/50
7/7 - 0s - loss: 1504.1218
Epoch 8/50
7/7 - 0s - loss: 1481.9623
Epoch 9/50
7/7 - 0s - loss: 1445.0662
Epoch 10/50
7/7 - 0s - loss: 1379.2318
Epoch 11/50
7/7 - 0s - loss: 1250.9802
Epoch 12/50
7/7 - 0s - loss: 1021.2416
Epoch 13/50
7/7 - 0s - loss: 701.8324
Epoch 14/50
7/7 - 0s - loss: 404.0992
Epoch 15/50
7/7 - 0s - loss: 319.5669
Epoch 16/50
7/7 - 0s - loss: 328.8557
Epoch 17/50
7/7 - 0s - loss: 298.1119
Epoch 18/50
7/7 - 0s - loss: 289.8801
Epoch 19/50
7/7 - 0s - loss: 289.3434
Epoch 20/50
7/7 - 0s - loss: 281.9050
Epoch 21/50
7/7 - 0s - loss: 276.5587
Epoch 22/50
7/7 - 0s - loss: 275.7686
Epoch 23/50
7/7 - 0s - loss: 272.0133
Epoch 24/50
7/7 - 0s - loss: 269.3776
Epoch 25/50
7/7 - 0s - loss: 268.4137
Epoch 26/50
7/7 - 0s - loss: 266.1645
Epoch 27/

Model evaluation

In [332]:
loss = model.evaluate(X_test, y_test)
y_hat = model.predict(X_test)
loss

7/7 [==============================] - 0s 1ms/step - loss: 223.6200


223.62002563476562

Create a list of mean squared errors and  the standard deviation of the mean squared errors


In [ ]:
total_mean_squared_errors = 50
mean_squared_errors = []
for i in range(0,total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.2, random_state=4)
    model.fit(X_train, y_train, epochs=50, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_hat = model.predict(X_test)
    errors = mean_squared_error(y_test, y_hat)
    mean_squared_errors.append(errors)
    
mean = np.mean(np.array(mean_squared_errors))
STD = np.std(mean_squared_errors)
print("Mean: ",str(mean), "Standard Deviation:",str(STD))


MSE 1: 80.21095275878906
MSE 2: 57.82047653198242
MSE 3: 52.49825668334961
MSE 4: 43.34389114379883
MSE 5: 43.0840950012207
MSE 6: 42.62962341308594
MSE 7: 43.13312911987305
MSE 8: 42.473812103271484
MSE 9: 44.44046401977539
MSE 10: 42.724571228027344
MSE 11: 41.933528900146484
MSE 12: 41.40144348144531
MSE 13: 41.14154052734375
MSE 14: 41.84152603149414
MSE 15: 46.95902633666992
MSE 16: 40.59736251831055
MSE 17: 44.42749786376953
MSE 18: 45.03876876831055
MSE 19: 42.263729095458984
MSE 20: 41.98814010620117
MSE 21: 43.26201248168945
MSE 22: 41.754878997802734
MSE 23: 43.955841064453125
MSE 24: 45.50306701660156
MSE 25: 48.2054557800293
MSE 26: 42.65309143066406
MSE 27: 41.98141098022461
MSE 28: 41.98827362060547
MSE 29: 41.81259536743164
MSE 30: 41.625431060791016
MSE 31: 42.984745025634766
MSE 32: 43.25538635253906
MSE 33: 41.51741409301758
MSE 34: 42.0477180480957
MSE 35: 46.45437240600586
MSE 36: 42.855892181396484
MSE 37: 43.65380096435547
MSE 38: 43.81819534301758
